# Introduction to Natural Language Processing (NLP) in TensorFlow

### Word Embeddings

Word embeddings, or word vectors, provide a way of mapping words from a vocabulary into a low-dimensional space, where words with similar meanings are close together. Let's play around with a set of pre-trained word vectors, to get used to their properties. There exist many sets of pretrained word embeddings; here, we use ConceptNet Numberbatch, which provides a relatively small download in an easy-to-work-with format (h5).

In [106]:
# Download word vectors
from urllib.request import urlretrieve
import os
if not os.path.isfile('mini.h5'):
    print("Downloading Conceptnet Numberbatch word embeddings...")
    conceptnet_url = 'http://conceptnet.s3.amazonaws.com/precomputed-data/2016/numberbatch/17.06/mini.h5'
    urlretrieve(conceptnet_url, 'mini.h5')

To read an `h5` file, we'll need to use the `h5py` package. Below, we use the package to open the `mini.h5` file we just downloaded. We extract from the file a list of utf-8-encoded words, as well as their $300$-dimensional vectors.

In [107]:
# Load the file and pull out words and embeddings
import h5py
with h5py.File('mini.h5', 'r') as f:
    all_words = [word.decode('utf-8') for word in f['mat']['axis1'][:]]
    all_embeddings = f['mat']['block0_values'][:]
    
print("all_words dimensions: {0}".format(len(all_words)))
print("all_embeddings dimensions: {0}".format(all_embeddings.shape))

print(all_words[10000])

all_words dimensions: 362891
all_embeddings dimensions: (362891, 300)
/c/de/lande


Now, `all_words` is a list of $V$ strings (what we call our *vocabulary*), and `all_embeddings` is a $V \times 300$ matrix. The strings are of the form `/c/language_code/word`—for example, `/c/en/cat` and `/c/es/gato`.

We are interested only in the English words. We use Python list comprehensions to pull out the indices of the English words, then extract just the English words (stripping the six-character `/c/en/` prefix) and their embeddings.

In [108]:
# Restrict our vocabulary to just the English words
english_words = [word[6:] for word in all_words if word.startswith('/c/en/')]
english_word_indices = [i for i, word in enumerate(all_words) if word.startswith('/c/en/')]
english_embeddings = all_embeddings[english_word_indices]

print("all_words dimensions: {0}".format(len(english_words)))
print("all_embeddings dimensions: {0}".format(english_embeddings.shape))

print(english_words[10000])

all_words dimensions: 150875
all_embeddings dimensions: (150875, 300)
bajillion


We want to look up words easily, so we create a dictionary that maps us from a word to its index in the word embeddings matrix.

In [109]:
index = {word: i for i, word in enumerate(english_words)}

Each word has an associated 300-dimensional embedding, which we can directly show by calling the word embeddings with that dictionary word.

In [110]:
our_word='cat'
our_word_index=index['cat']
our_embedding=english_embeddings[our_word_index]
print(our_word,'has word index',our_word_index)
print(our_word, 'has word embedding',our_embedding)

cat has word index 21398
cat has word embedding [  0   0   2  -5   1   0  -1  -3   3  -2   2   1   4   0  -3   0  -1  -4
   0   1   0  -2   0  -2  -2   3   1  -2  -2   3  -2   1   0   0  -3   1
  -4   6  -1   2  -4   4  -3   0  -2  -3  -1  -5   5  -6   2   2  -3   0
  -1   4  -1   2   2   1   3   1   3  -1  -8  -3   2   0   1   4  -1  -5
   2   0  -5   1   0  -7  -4   2   3   4   3   3  -4   8   0   4  -2   0
   3   1  -4   0   3   0  -2   0  -1  -8   0   0   2   0   2   4   7   3
  -2   3  -3   3   2  -3  -5   0  -2   2   0  -3  10  -2   0  -3  -2  -5
   4   0   0   3   2  -1  -1   5  -4  -2   0  -2  -8   0  -4   4   9  -6
   5   4  -7  -1  -2  -3   2   4   0  -1  -6  -2  -4  -4  -2   0   4  -5
   0   2   0   4   1   3   7  -3   4   0   6   0 -12  -4   0  -2  -6   1
   6  -5   4   0   2  -6   2  -2   2   3  10   0   0   4   4  -2   0  -2
   0   3   1   0  -9   3   3  -6   0   0   0  -8   1   5   2  -2   0   1
  -3   4  -5  -5   3   1   4  -4   0   2  -2  -1  -4  -1   6  -2   3   3
  -

The magnitude of a word vector is less important than its direction; the magnitude can be thought of as representing frequency of use, independent of the semantics of the word. 
Here, we will be interested in semantics, so we *normalize* our vectors, dividing each by its length. 
The result is that all of our word vectors are length 1, and as such, lie on a unit circle. 
The dot product of two vectors is proportional to the cosine of the angle between them, and provides a measure of similarity (the bigger the cosine, the smaller the angle).

<img src="Figures/cosine_similarity.png" alt="cosine" style="width: 500px;"/>
<center>Figure adapted from *[Mastering Machine Learning with Spark 2.x](https://www.safaribooksonline.com/library/view/mastering-machine-learning/9781785283451/ba8bef27-953e-42a4-8180-cea152af8118.xhtml)*</center>

In [111]:
import numpy as np

norms = np.linalg.norm(english_embeddings, axis=1)
normalized_embeddings = english_embeddings.astype('float32') / norms.astype('float32').reshape([-1, 1])

Now we are ready to measure the similarity between pairs of words. We use numpy to take dot products.

In [112]:
def similarity_score(w1, w2):
    score = np.dot(normalized_embeddings[index[w1], :], normalized_embeddings[index[w2], :])
    return score

In [113]:
# A word is as similar with itself as possible:
print('cat\tcat\t', similarity_score('cat', 'cat'))

# Closely related words still get high scores:
print('cat\tfeline\t', similarity_score('cat', 'feline'))
print('cat\tdog\t', similarity_score('cat', 'dog'))

# Unrelated words, not so much
print('cat\tmoo\t', similarity_score('cat', 'moo'))
print('cat\tfreeze\t', similarity_score('cat', 'freeze'))

# Antonyms are still considered related, sometimes more so than synonyms
print('antonyms\topposites\t', similarity_score('antonym', 'opposite'))
print('antonyms\tsynonyms\t', similarity_score('antonym', 'synonym'))

cat	cat	 1.0
cat	feline	 0.8199548
cat	dog	 0.590724
cat	moo	 0.0039538266
cat	freeze	 -0.030225186
antonyms	opposites	 0.3941065
antonyms	synonyms	 0.46883982


We can also find, for instance, the most similar words to a given word.

In [114]:
def closest_to_vector(v, n):
    all_scores = np.dot(normalized_embeddings, v)
    best_words = map(lambda i: english_words[i], reversed(np.argsort(all_scores)))
    return [next(best_words) for _ in range(n)]

def most_similar(w, n):
    return closest_to_vector(normalized_embeddings[index[w], :], n)

In [115]:
print(most_similar('cat', 10))
print(most_similar('dog', 10))
print(most_similar('duke', 10))

['cat', 'humane_society', 'kitten', 'feline', 'colocolo', 'cats', 'kitty', 'maine_coon', 'housecat', 'sharp_teeth']
['dog', 'dogs', 'wire_haired_dachshund', 'doggy_paddle', 'lhasa_apso', 'good_friend', 'puppy_dog', 'bichon_frise', 'woof_woof', 'golden_retrievers']
['duke', 'dukes', 'duchess', 'duchesses', 'ducal', 'dukedom', 'duchy', 'voivode', 'princes', 'prince']


**Can you explain the following similarity scores?**

In [116]:
similarity_score("sit", "sits")

0.8478777

In [117]:
similarity_score("want", "wants")

0.858501

In [118]:
similarity_score("sleep", "sleeps")

0.8664926

In [119]:
similarity_score("leave", "leaves")

0.42647985

**Can you find a polysemous word — a word with multiple meanings — so that the list of the top 10 most related words contains words that aren't themselves related to one another?**

In [120]:
print(most_similar('leave', 10))

['leave', 'leaving', 'come_away', 'depart', 'beleave', 'go_forth', 'quit', 'go_away', 'vacate', 'departing']


In [121]:
print(most_similar('leaves', 10))

['leaves', 'leaf', 'foliage', 'banana_leaf', 'leaflike', 'leaved', 'betel_leaf', 'thai_basil', 'oak_trees', 'aspidistra']


In [122]:
print(most_similar('solution', 10))

['solution', 'solutions', 'aqueous_solution', 'subproblem', 'solve', 'aqueous_phase', 'virtuous_circle', 'solves', 'resolvent', 'solving']


We can also use `closest_to_vector` to find words "nearby" vectors that we create ourselves. This allows us to solve analogies. For example, in order to solve the analogy "man : brother :: woman : ?", we can compute a new vector `brother - man + woman`: the meaning of brother, minus the meaning of man, plus the meaning of woman. We can then ask which words are closest, in the embedding space, to that new vector.

In [123]:
def solve_analogy(a1, b1, a2):
    b2 = normalized_embeddings[index[b1], :] - normalized_embeddings[index[a1], :] + normalized_embeddings[index[a2], :]
    return closest_to_vector(b2, 5)
def print_analogy(a1, b1,a2):
    closest_words=solve_analogy(a1,b1,a2)
    print("{0}:{1} as {2}:?".format(a1,b1,a2))
    print("Best guesses are: {}".format(closest_words))
    return None

In [124]:
print_analogy("man", "brother", "woman")
print_analogy("man", "husband", "woman")
print_analogy("spain", "madrid", "france")
print_analogy("dog", "golden_retriever", "cat")

man:brother as woman:?
Best guesses are: ['sister', 'brother', 'sisters', 'kid_sister', 'younger_brother']
man:husband as woman:?
Best guesses are: ['wife', 'husband', 'husbands', 'spouse', 'wifes']
spain:madrid as france:?
Best guesses are: ['paris', 'france', 'le_havre', 'in_france', 'montmartre']
dog:golden_retriever as cat:?
Best guesses are: ['cat', 'maine_coon', 'kitten', 'tabby', 'kitty']


**Note that the new vector, `b2`, is not normalized. Does this matter?  Why or why not?**

These three results are quite good, but in general, the results of these analogies can be disappointing. Try experimenting with other analogies, and see if you can think of ways to get around the problems you notice (i.e., modifications to the solve_analogy algorithm).

### Using word embeddings in deep models
Word embeddings are fun to play around with, but their primary use is that they allow us to think of words as existing in a continuous, Euclidean space; we can then use an existing arsenal of techniques for machine learning with continuous numerical data (like logistic regression or neural networks) to process text.

Let's take a look at an especially simple version of this. We'll perform *sentiment analysis* on a set of movie reviews: in particular, we will attempt to classify a movie review as positive or negative based on its text.

We will use a simplified version of [Simple Word Embedding Model](http://people.ee.duke.edu/~lcarin/acl2018_swem.pdf) (SWEM, Shen et al. 2018) to do so. We will represent a review as the *mean* of the embeddings of the words in the review (SWEM would also update the word embeddings). Then we'll train a three-layer MLP (a neural network) to classify the review as positive or negative.

Download the `movie-simple.txt` file from Google Classroom into this directory. Each line of that file contains 

1. the numeral 0 (for negative) or the numeral 1 (for positive), followed by
2. a tab (the whitespace character), and then
3. the review itself.

In [125]:
import string
remove_punct=str.maketrans('','',string.punctuation)

# This function converts a line of our data file into
# a tuple (x, y), where x is 300-dimensional representation
# of the words in a review, and y is its label.
def convert_line_to_example(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [normalized_embeddings[index[w]] for w in words
                  if w in index]
    
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return {'x': x, 'y': y}

# Apply the function to each line in the file.
with open("Resources/movie-simple.txt", "r", encoding='utf-8', errors='ignore') as f:
    dataset = [convert_line_to_example(l) for l in f.readlines()]

In [126]:
len(dataset)

1411

Now that we have a dataset, let's shuffle it and do a train/test split. We use a quarter of the dataset for testing, 3/4 for training (but also ensure that we have a whole number of batches in our training set, to make the code nicer later).

In [127]:
import random
random.shuffle(dataset)

batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = 3*total_batches // 4 
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

Time to build our MLP in Tensorflow. We'll use placeholders for `X` and `y` as usual.

In [128]:
import tensorflow as tf

# Placeholders for input
X = tf.placeholder(tf.float32, [None, 300])
y = tf.placeholder(tf.float32, [None, 1])

# Three-layer MLP
h1 = tf.keras.layers.Dense(100, activation='relu')(X)
h2 = tf.keras.layers.Dense(20, activation='relu')(h1)
logits = tf.keras.layers.Dense(1)(h2)
probabilities = tf.sigmoid(logits)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(probabilities), y), tf.float32))

# Training
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

# Initialization of variables
init_op = tf.global_variables_initializer()

We can now begin a session and train our model. We'll train for 250 epochs. When we're finished, we'll evaluate our accuracy on all the test data.

In [129]:
# Train
sess = tf.Session()
sess.run(init_op)

for epoch in range(250):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = [sample['x'] for sample in data]
        labels  = [sample['y'] for sample in data]
        labels = np.array(labels).reshape([-1,1])
        
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        
    if epoch % 10 == 0:
        print("Epoch: {0} \t Loss: {1} \t Acc: {2}".format(epoch, l, acc))
    
    random.shuffle(train)
        
# Evaluate on test set
test_reviews = [sample['x'] for sample in test]
test_labels  = [sample['y'] for sample in test]
test_labels  = np.array(test_labels).reshape([-1, 1])

acc = sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
print("Final accuracy: {0}".format(acc))

Epoch: 0 	 Loss: 0.6889349222183228 	 Acc: 0.6200000047683716
Epoch: 10 	 Loss: 0.6749433875083923 	 Acc: 0.5899999737739563
Epoch: 20 	 Loss: 0.6782694458961487 	 Acc: 0.550000011920929
Epoch: 30 	 Loss: 0.6783126592636108 	 Acc: 0.5199999809265137
Epoch: 40 	 Loss: 0.6950281262397766 	 Acc: 0.44999998807907104
Epoch: 50 	 Loss: 0.6358800530433655 	 Acc: 0.6399999856948853
Epoch: 60 	 Loss: 0.6386836767196655 	 Acc: 0.6399999856948853
Epoch: 70 	 Loss: 0.604857325553894 	 Acc: 0.6700000166893005
Epoch: 80 	 Loss: 0.5425114631652832 	 Acc: 0.8399999737739563
Epoch: 90 	 Loss: 0.5198315978050232 	 Acc: 0.800000011920929
Epoch: 100 	 Loss: 0.4235253632068634 	 Acc: 0.8999999761581421
Epoch: 110 	 Loss: 0.3741091787815094 	 Acc: 0.8999999761581421
Epoch: 120 	 Loss: 0.29639628529548645 	 Acc: 0.9100000262260437
Epoch: 130 	 Loss: 0.2626670300960541 	 Acc: 0.9300000071525574
Epoch: 140 	 Loss: 0.21935081481933594 	 Acc: 0.9300000071525574
Epoch: 150 	 Loss: 0.20941413938999176 	 Acc: 0.920

We can now examine what our model has learned, seeing how it responds to word vectors for different words:

In [130]:
# Check some words
words_to_test = ["exciting", "hated", "boring", "loved", "extremely", "rather", "quite"]

for word in words_to_test:
    print(word, sess.run(probabilities, feed_dict={X: normalized_embeddings[index[word]].reshape(1, 300)}))

exciting [[0.9999815]]
hated [[3.004384e-08]]
boring [[5.114875e-06]]
loved [[0.9999999]]
extremely [[0.59495217]]
rather [[0.00566179]]
quite [[0.5447439]]


Try some words of your own!

In [131]:
sess.close()

This model works great for such a simple dataset, but does a little less well on something more complex. `movie-pang02.txt`, for instance, has 2000 longer, more complex movie reviews. It's in the same format as our simple dataset. On those longer reviews, this model achieves only 60-80% accuracy. (Increasing the number of epochs to, say, 1000, does help.)

To show this in practice, this is a condensed version of the same code above working on this second dataset.

In [132]:
# Apply the function to each line in the file.
with open("Resources/movie-pang02.txt", "r",encoding='utf-8') as f:
    dataset = [convert_line_to_example(l) for l in f.readlines()]
import random
random.shuffle(dataset)
batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]
sess = tf.Session()
initialize_all = tf.global_variables_initializer() 
sess.run(initialize_all)
for epoch in range(250):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = [sample['x'] for sample in data]
        labels  = [sample['y'] for sample in data]
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
    if epoch % 10 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)

# Evaluate on test set
test_reviews = [sample['x'] for sample in test]
test_labels  = [sample['y'] for sample in test]
test_labels = np.array(test_labels).reshape([-1, 1])
acc = sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
print("Final accuracy:", acc)
sess.close()

Epoch 0 Loss 0.69324297 Acc 0.47
Epoch 10 Loss 0.6920747 Acc 0.67
Epoch 20 Loss 0.6920169 Acc 0.54
Epoch 30 Loss 0.6918787 Acc 0.56
Epoch 40 Loss 0.6914005 Acc 0.57
Epoch 50 Loss 0.6909299 Acc 0.56
Epoch 60 Loss 0.6924898 Acc 0.48
Epoch 70 Loss 0.6917926 Acc 0.58
Epoch 80 Loss 0.69205946 Acc 0.53
Epoch 90 Loss 0.6918338 Acc 0.49
Epoch 100 Loss 0.6907446 Acc 0.53
Epoch 110 Loss 0.69052535 Acc 0.6
Epoch 120 Loss 0.68922544 Acc 0.65
Epoch 130 Loss 0.688009 Acc 0.65
Epoch 140 Loss 0.68838364 Acc 0.62
Epoch 150 Loss 0.6877033 Acc 0.65
Epoch 160 Loss 0.68835264 Acc 0.54
Epoch 170 Loss 0.68397325 Acc 0.67
Epoch 180 Loss 0.6879108 Acc 0.6
Epoch 190 Loss 0.6832516 Acc 0.66
Epoch 200 Loss 0.6884147 Acc 0.55
Epoch 210 Loss 0.67951775 Acc 0.7
Epoch 220 Loss 0.68232423 Acc 0.6
Epoch 230 Loss 0.6805441 Acc 0.67
Epoch 240 Loss 0.6713426 Acc 0.72
Final accuracy: 0.596


We will not plan to go through Recurrent Neural Networks in detail, but below is some example code on RNNs to provide some guidance.

### Recurrent Neural Networks (RNNs)

In the context of deep learning, natural language is commonly modeled with Recurrent Neural Networks (RNNs).
RNNs pass the output of a neuron back to the input of the next time step of the same neuron.
These directed cycles in the RNN architecture gives them the ability to model temporal dynamics, making them particularly suited for modeling sequences (e.g. text).
We can visualize an RNN layer as follows:

<img src="Figures/basic_RNN.PNG" alt="basic_RNN" style="width: 80px;"/>
<center>Figure from *Understanding LSTMs*. https://colah.github.io/posts/2015-08-Understanding-LSTMs/</center>

We can unroll an RNN through time, making the sequence aspect of them more obvious:

<img src="Figures/unrolled_RNN.PNG" alt="basic_RNN" style="width: 400px;"/>
<center>Figure from *Understanding LSTMs*. https://colah.github.io/posts/2015-08-Understanding-LSTMs/</center>

#### RNNs in TensorFlow
How would we implement an RNN in TensorFlow? Given the different forms of RNNs, there are quite a few ways, but we'll stick to a simple one. 

When we are dealing with a Recurrent Neural Network, we can have each word be a separate input to the network.  Given our word embeddings, that will be given by a matrix. The preprocessing pipeline will be slightly different than it was before.

In [133]:
# This function converts a line of our data file into
# a tuple (x, y, w), where x is 300-dimensional representation
# of the words in a review,  y is its label, and w is the concatenated word embeddings
def convert_line_to_example_rnn(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [normalized_embeddings[index[w]] for w in words
                  if w in index]
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return {'x': x, 'y': y, 'w':embeddings, 'text':line[2:]}

In [134]:
with open("Resources/movie-simple.txt", "r",encoding='utf-8') as f:
    dataset = [convert_line_to_example_rnn(l) for l in f.readlines()]
import random
random.seed(200)
random.shuffle(dataset)
batch_size = 1
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

In [135]:
print(train[0]['text'])
print(train[0]['y'])
print(train[0]['x'].shape)
print(np.array(train[0]['w']).reshape([1,-1,300]).shape)

The DaVinci Code and Mission Impossible 3 are AWESOME.

1
(300,)
(1, 9, 300)


Applying an RNN to the text reviews, starting with the easier data.

In [136]:
tf.reset_default_graph()
# sizes
n_steps = None
n_inputs = 300
n_neurons = 5
# Build RNN
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y= tf.placeholder(tf.float32, [None, 1])
basic_cell = tf.contrib.rnn.BasicRNNCell(n_neurons,activation=tf.nn.tanh)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
last_cell_output=outputs[:,-1,:]
y_=tf.layers.dense(last_cell_output,1)

In [137]:
# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(y_)), y), tf.float32))

# Training
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [138]:
initialize_all = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(initialize_all)
for epoch in range(10):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = np.array([sample['w'] for sample in data]).reshape([batch_size,-1,300])
        labels  = np.array([sample['y'] for sample in data]).reshape([batch_size,1])
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        if (batch+1) % 500 == 0:
            print("batch", batch, "Loss", l, "Acc", acc)
    if epoch % 1 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)


batch 499 Loss 0.6888297 Acc 1.0
batch 999 Loss 0.15129247 Acc 1.0
Epoch 0 Loss 1.8025193 Acc 0.0
batch 499 Loss 0.11805359 Acc 1.0
batch 999 Loss 0.057079393 Acc 1.0
Epoch 1 Loss 0.08673588 Acc 1.0
batch 499 Loss 0.029195866 Acc 1.0
batch 999 Loss 0.090121076 Acc 1.0
Epoch 2 Loss 0.28285712 Acc 1.0
batch 499 Loss 0.011758866 Acc 1.0
batch 999 Loss 0.11506314 Acc 1.0
Epoch 3 Loss 1.4820299 Acc 0.0
batch 499 Loss 0.00388214 Acc 1.0
batch 999 Loss 0.015506666 Acc 1.0
Epoch 4 Loss 0.0733734 Acc 1.0
batch 499 Loss 0.029356554 Acc 1.0
batch 999 Loss 0.047062576 Acc 1.0
Epoch 5 Loss 0.011984888 Acc 1.0
batch 499 Loss 0.12030737 Acc 1.0
batch 999 Loss 0.16438444 Acc 1.0
Epoch 6 Loss 0.01008274 Acc 1.0
batch 499 Loss 0.008522104 Acc 1.0
batch 999 Loss 2.229559 Acc 0.0
Epoch 7 Loss 0.005230967 Acc 1.0
batch 499 Loss 0.13257532 Acc 1.0
batch 999 Loss 0.0014965261 Acc 1.0
Epoch 8 Loss 0.0025378007 Acc 1.0
batch 499 Loss 0.004843535 Acc 1.0
batch 999 Loss 0.93674695 Acc 0.0
Epoch 9 Loss 0.00598276

In [139]:
# Evaluate on test set
test_acc=0
n=0
for sample in test:
    test_reviews = np.array([sample['w'] ]).reshape([1,-1,300])
    test_labels  = np.array([sample['y']]).reshape([1,1])
    test_labels = np.array(test_labels).reshape([-1, 1])
    test_acc += sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
    n+=1
acc=test_acc/n 
print("Final accuracy:", acc)

Final accuracy: 0.9773371104815864


In [140]:
sess.close()

Switching to an LSTM-based neural network is fairly easy in code.

In [141]:
tf.reset_default_graph()
# sizes
n_steps = None
n_inputs = 300
n_neurons = 5
# Build RNN
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y= tf.placeholder(tf.float32, [None, 1])
basic_cell = tf.contrib.rnn.LSTMCell(n_neurons,activation=tf.nn.tanh)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
last_cell_output=outputs[:,-1,:]
y_=tf.layers.dense(last_cell_output,1)

In [142]:
# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(y_)), y), tf.float32))

# Training
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)

In [143]:
initialize_all = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initialize_all)
for epoch in range(10):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = np.array([sample['w'] for sample in data]).reshape([batch_size,-1,300])
        labels  = np.array([sample['y'] for sample in data]).reshape([batch_size,1])
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        if (batch+1) % 500 == 0:
            print("batch", batch, "Loss", l, "Acc", acc)
    if epoch % 1 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)

batch 499 Loss 0.48607454 Acc 1.0
batch 999 Loss 0.22863635 Acc 1.0
Epoch 0 Loss 0.25951394 Acc 1.0
batch 499 Loss 0.101040214 Acc 1.0
batch 999 Loss 0.05657885 Acc 1.0
Epoch 1 Loss 0.05204625 Acc 1.0
batch 499 Loss 0.09560171 Acc 1.0
batch 999 Loss 0.071655296 Acc 1.0
Epoch 2 Loss 0.059827775 Acc 1.0
batch 499 Loss 0.058376465 Acc 1.0
batch 999 Loss 0.055644244 Acc 1.0
Epoch 3 Loss 0.16751455 Acc 1.0
batch 499 Loss 0.11454698 Acc 1.0
batch 999 Loss 0.013678055 Acc 1.0
Epoch 4 Loss 0.0361196 Acc 1.0
batch 499 Loss 0.4248932 Acc 1.0
batch 999 Loss 0.020090805 Acc 1.0
Epoch 5 Loss 0.0067319837 Acc 1.0
batch 499 Loss 0.024903579 Acc 1.0
batch 999 Loss 0.0086562745 Acc 1.0
Epoch 6 Loss 0.00056374667 Acc 1.0
batch 499 Loss 0.040345907 Acc 1.0
batch 999 Loss 0.005661821 Acc 1.0
Epoch 7 Loss 0.029291177 Acc 1.0
batch 499 Loss 0.008131191 Acc 1.0
batch 999 Loss 0.0034976238 Acc 1.0
Epoch 8 Loss 0.0014228114 Acc 1.0
batch 499 Loss 0.017069545 Acc 1.0
batch 999 Loss 0.0028547666 Acc 1.0
Epoch 9 

In [144]:
# Evaluate on test set
test_acc=0
n=0
for sample in test:
    test_reviews = np.array([sample['w'] ]).reshape([1,-1,300])
    test_labels  = np.array([sample['y']]).reshape([1,1])
    test_labels = np.array(test_labels).reshape([-1, 1])
    test_acc += sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
    n+=1
acc=test_acc/n 
print("Final accuracy:", acc)

Final accuracy: 0.9858356940509915


In [145]:
sess.close()

Finally, swapping out to a more complex dataset.

In [146]:
with open("Resources/movie-pang02.txt", "r") as f:
    dataset = [convert_line_to_example_rnn(l) for l in f.readlines()]
import random
random.seed(42)
random.shuffle(dataset)
batch_size = 1
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

By examining one of the reviews, we can see that these are much more complex reviews, and requiring greater nuance to analyze.

In [147]:
print(train[0]['text'])
print(train[0]['y'])
print(train[0]['x'].shape)
print(np.array(train[0]['w']).reshape([1,-1,300]).shape)

 the central focus of michael winterbottom s   welcome to sarajevo   is sarajevo itself   the city under siege   and its different effect on the characters unfortunate enough to be stuck there    it proves the backdrop for a stunningly realized story which refreshingly strays from mythic portents     platoon       racial tumultuosness   the risible   the walking dead     or a tinge of schmaltziness     schindler s list        the two leads   stephen dillane as a reporter and emira nusevic as an orphan with a plight few can identify with   are extremely believable   not one moment with them involved rings false    the question is not what went right    the question is what went wrong    for one   the film fails to provide a political overview of the war as it progresses   the dillane characters reports an american plane departing from sarajevo as it departs   and that s about it        the assortment of high profile supporting actors   ranging from woody harrelson as a yankee reporter  

In [148]:
len(train)

1500

We can retrain on the same LSTM that we were using before. Note that this is _much_ slower because the RNN now has to deal with much longer sequences.

In [149]:
initialize_all = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initialize_all)
for epoch in range(10):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = np.array([sample['w'] for sample in data]).reshape([batch_size,-1,300])
        labels  = np.array([sample['y'] for sample in data]).reshape([batch_size,1])
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        if (batch+1) % 500 == 0:
            print("batch", batch, "Loss", l, "Acc", acc)
    if epoch % 1 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)

batch 499 Loss 0.63374853 Acc 1.0
batch 999 Loss 0.65800625 Acc 1.0
batch 1499 Loss 0.6028506 Acc 1.0
Epoch 0 Loss 0.6028506 Acc 1.0
batch 499 Loss 0.6138419 Acc 1.0
batch 999 Loss 0.9523419 Acc 0.0
batch 1499 Loss 0.8965019 Acc 0.0
Epoch 1 Loss 0.8965019 Acc 0.0
batch 499 Loss 0.6895683 Acc 1.0
batch 999 Loss 0.3232401 Acc 1.0
batch 1499 Loss 0.5422541 Acc 1.0
Epoch 2 Loss 0.5422541 Acc 1.0
batch 499 Loss 0.19960187 Acc 1.0
batch 999 Loss 1.1023433 Acc 0.0
batch 1499 Loss 0.4265583 Acc 1.0
Epoch 3 Loss 0.4265583 Acc 1.0
batch 499 Loss 0.8223858 Acc 0.0
batch 999 Loss 0.30732954 Acc 1.0
batch 1499 Loss 0.30599624 Acc 1.0
Epoch 4 Loss 0.30599624 Acc 1.0
batch 499 Loss 0.9944975 Acc 0.0
batch 999 Loss 0.92760015 Acc 0.0
batch 1499 Loss 1.3472576 Acc 0.0
Epoch 5 Loss 1.3472576 Acc 0.0
batch 499 Loss 0.25653654 Acc 1.0
batch 999 Loss 0.28753564 Acc 1.0
batch 1499 Loss 0.21350908 Acc 1.0
Epoch 6 Loss 0.21350908 Acc 1.0
batch 499 Loss 1.2260671 Acc 0.0
batch 999 Loss 0.9523636 Acc 0.0
batch 

In [150]:
# Evaluate on test set
test_acc=0
n=0
for sample in test:
    test_reviews = np.array([sample['w'] ]).reshape([1,-1,300])
    test_labels  = np.array([sample['y']]).reshape([1,1])
    test_labels = np.array(test_labels).reshape([-1, 1])
    test_acc += sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
    n+=1
acc=test_acc/n 
print("Final accuracy:", acc)

Final accuracy: 0.788


In [151]:
sess.close()

This final accuracy is not great, but it is _much higher_ than what we got with the MLP approach.  Playing around with the settings can properly improve this more (this really isn't that many epochs).